# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [1]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [2]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

def get_llm_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """Get a response from the LLM based on the prompt"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
#testing
if __name__ == "__main__":
    print("Zero-shot:\n", get_llm_response(zero_shot_prompt))
    print("\nOne-shot:\n", get_llm_response(one_shot_prompt))
    print("\nFew-shot:\n", get_llm_response(few_shot_prompt))

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-06 18:48:21.955780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749235702.268325    2476 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749235702.363166    2476 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749235703.055628    2476 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000

NameError: name 'zero_shot_prompt' is not defined

## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [2]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# Define a function to format prompts for each strategy
def create_prompts(question, example_q, example_a, examples):
    """Creates prompts for zero-shot, one-shot, and few-shot strategies."""
    zero_shot_template = "Question: {question}\nAnswer:"
    one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

    few_shot_template = ""
    for ex_q, ex_a in examples:
        few_shot_template += f"Question: {ex_q}\nAnswer: {ex_a}\n\n"
    few_shot_template += "Question: {question}\nAnswer:"


    zero_shot_prompt = zero_shot_template.format(question=question)
    one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
    few_shot_prompt = few_shot_template.format(question=question)

    return {
        "zero_shot": zero_shot_prompt,
        "one_shot": one_shot_prompt,
        "few_shot": few_shot_prompt
    }

# TODO: Compare the different prompting strategies on these questions
# For each question:
# - Create prompts using each strategy
# - Get responses from the LLM
# - Store the results

results = {}
api_key = "YOUR_HUGGINGFACE_API_KEY" # Replace with your actual API key
for q in questions:
        print(f"\nTesting question: {q}")
        current_prompts = create_prompts(q, example_q, example_a, examples)

        results[q] = {}
        for strategy, prompt in current_prompts.items():
            print(f"  Strategy: {strategy}")
            response = get_llm_response(prompt, api_key=api_key)
            results[q][strategy] = response
            print(f"    Response: {response[:100]}...") # Print a snippet of the response

# Analyze and present the results
print("\n--- Comparison Results ---")
for q, strategy_results in results.items():
        print(f"\nQuestion: {q}")
        for strategy, response in strategy_results.items():
            print(f"  {strategy.capitalize()}: {response}")


Testing question: What foods should be avoided by patients with gout?


NameError: name 'example_q' is not defined

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [5]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?":
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?":
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?":
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?":
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?":
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
strategy_scores = {
    "zero_shot": [],
    "one_shot": [],
    "few_shot": []
}

print("\n--- Scoring Responses ---")
for q, strategy_results in results.items():
    print(f"\nQuestion: {q}")
    keywords = expected_keywords.get(q, [])
    if not keywords:
        print(f"  No keywords defined for this question.")
        continue

    for strategy, response in strategy_results.items():
        score = score_response(response, keywords)
        strategy_scores[strategy].append(score)
        print(f"  {strategy.capitalize()} Score: {score:.2f}")

# Calculate and print average scores for each strategy
average_scores = {
    strategy: sum(scores) / len(scores) if scores else 0
    for strategy, scores in strategy_scores.items()
}

print("\n--- Average Scores per Strategy ---")
for strategy, avg_score in average_scores.items():
    print(f"{strategy.capitalize()}: {avg_score:.2f}")

# Determine which strategy performs best overall
best_strategy = max(average_scores, key=average_scores.get)
print(f"\nBest performing strategy overall: {best_strategy.capitalize()}")


--- Scoring Responses ---

Question: What foods should be avoided by patients with gout?
  Zero_shot Score: 0.00
  One_shot Score: 0.00
  Few_shot Score: 0.00

Question: What medications are commonly prescribed for gout?
  Zero_shot Score: 0.00
  One_shot Score: 0.00
  Few_shot Score: 0.00

Question: How can gout flares be prevented?
  Zero_shot Score: 0.00
  One_shot Score: 0.00
  Few_shot Score: 0.00

Question: Is gout related to diet?
  Zero_shot Score: 0.00
  One_shot Score: 0.00
  Few_shot Score: 0.00

Question: Can gout be cured permanently?
  Zero_shot Score: 0.00
  One_shot Score: 0.00
  Few_shot Score: 0.00

--- Average Scores per Strategy ---
Zero_shot: 0.00
One_shot: 0.00
Few_shot: 0.00

Best performing strategy overall: Zero_shot


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [8]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""

# Initialize output_string
output_string = ""

# Add header and raw responses 
output_string += "# Prompt Engineering Results\n\n"

for q, strategy_results in results.items():
    output_string += f"## Question: {q}\n\n"
    for strategy, response in strategy_results.items():
        output_string += f"### {strategy.capitalize()} response:\n"
        output_string += f"{response}\n\n"
    output_string += "--------------------------------------------------\n\n"


# Add scores section
output_string += "## Scores\n\n"
output_string += "```\n"

# Header row
output_string += "question,zero_shot,one_shot,few_shot\n"

# Data rows for each question
for i, q in enumerate(questions):
    # Simple slugify the question for the row header
    q_slug = q.lower().replace(' ', '_').replace('?','').replace(',','')
    output_string += f"{q_slug},"
    scores_for_q = []
    for strategy in ["zero_shot", "one_shot", "few_shot"]:
        # Find the score for this specific question and strategy
        if i < len(strategy_scores[strategy]):
             scores_for_q.append(f"{strategy_scores[strategy][i]:.2f}")
        else:
             scores_for_q.append("N/A") # Handle cases where scoring might not have happened
    output_string += ",".join(scores_for_q) + "\n"

# Add average row
average_scores_list = [f"{average_scores['zero_shot']:.2f}", f"{average_scores['one_shot']:.2f}", f"{average_scores['few_shot']:.2f}"]
output_string += f"average,{','.join(average_scores_list)}\n"

# Add best method row
best_strategy = max(average_scores, key=average_scores.get)
output_string += f"best_method,{best_strategy}\n"

output_string += "```\n"

# TODO: Save your results to results/part_3/prompting_results.txt
# Create the directory if it doesn't exist
import os
results_dir = "results/part_3"
os.makedirs(results_dir, exist_ok=True)

# Write the output string to the file
file_path = os.path.join(results_dir, "prompting_results.txt")
with open(file_path, "w") as f:
    f.write(output_string)

print(f"Results saved to {file_path}")

Results saved to results/part_3/prompting_results.txt


## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method